In [47]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit, train_test_split
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [105]:
df=pd.read_csv("dataset.csv")

In [107]:
df.columns

Index(['Unnamed: 0', 'activity', 'appCat.builtin', 'appCat.communication',
       'appCat.entertainment', 'appCat.finance', 'appCat.game',
       'appCat.office', 'appCat.other', 'appCat.social', 'appCat.travel',
       'appCat.unknown', 'appCat.utilities', 'appCat.weather', 'call',
       'circumplex.arousal', 'circumplex.valence', 'mood', 'screen', 'sms',
       'id', 'day', 'target_mood'],
      dtype='object')

In [109]:
#sort by data 
df['day'] = pd.to_datetime(df['day'])
df = df.sort_values(by='day')

In [111]:
len(df['day'].unique())

86

In [113]:
df = df.drop(columns=['Unnamed: 0','id','day'])

In [115]:
#seperate 80/20
train_df, temp_df = train_test_split(df, test_size=0.2, shuffle=False)

#seperate 20 into 50/50
val_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=False)

print("Train shape:", train_df.shape)
print("Validation shape:", val_df.shape)
print("Test shape:", test_df.shape)

Train shape: (935, 20)
Validation shape: (117, 20)
Test shape: (117, 20)


In [117]:
X_train = train_df.drop(['target_mood'], axis=1)
y_train = train_df['target_mood']

X_val = val_df.drop(['target_mood'], axis=1)
y_val = val_df['target_mood']

X_test = test_df.drop(['target_mood'], axis=1)
y_test = test_df['target_mood']

In [121]:
#validate that time series is correct --> I checked this but now this dindt work because days is remvmove for the model 
"""print(train_df['day'].min())
print(val_df['day'].min())
print(test_df['day'].min())"""

"print(train_df['day'].min())\nprint(val_df['day'].min())\nprint(test_df['day'].min())"

In [125]:
# define model
model = Sequential()

#shape is based on number of unique timestamps (86) and number of features (22)
model.add(LSTM(units=50, input_shape=(86, 19)))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_data=(X_val, y_val))



C:\Users\youpz\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_5 (LSTM)                   │ (None, 50)             │        14,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,051 (54.89 KB)

 Trainable params: 14,051 (54.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(None, 19), dtype=float32). Expected shape (None, 86, 19), but input has incompatible shape (None, 19)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 19), dtype=float32)
  • training=True
  • mask=None

In [ ]:
#evaluating of the code
test_loss = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)